In [33]:
import numpy as np
import pandas as pd

dt1 = pd.read_csv('pg1661-1.txt',"\t", header=None)


In [34]:
print(dt1)

                                                      0
0     Project Gutenberg's The Adventures of Sherlock...
1     This eBook is for the use of anyone anywhere a...
2     almost no restrictions whatsoever.  You may co...
3     re-use it under the terms of the Project Guten...
4        with this eBook or online at www.gutenberg.net
...                                                 ...
7018                           http://www.gutenberg.net
7019  This Web site includes information about Proje...
7020  including how to make donations to the Project...
7021  Archive Foundation, how to help produce our ne...
7022  subscribe to our email newsletter to hear abou...

[7023 rows x 1 columns]


In [35]:
dt2 = pd.read_csv('pg31100.txt',"\t", header=None)
print(dt2)

                                                       0
0      Project Gutenberg's The Complete Works of Jane...
1      This eBook is for the use of anyone anywhere a...
2      almost no restrictions whatsoever.  You may co...
3      re-use it under the terms of the Project Guten...
4         with this eBook or online at www.gutenberg.org
...                                                  ...
56183                           http://www.gutenberg.org
56184  This Web site includes information about Proje...
56185  including how to make donations to the Project...
56186  Archive Foundation, how to help produce our ne...
56187  subscribe to our email newsletter to hear abou...

[56188 rows x 1 columns]


In [36]:
# Data pre-processing : clear out all punctuations in our data
import string
punctuation_string = string.punctuation
n1 = dt1.shape[0]
n2 = dt2.shape[0]
for j in range(n1):
    for i in punctuation_string:
        dt1.loc[j][0] = dt1.loc[j][0].replace(i, '')
for k in range(n2):
    for i in punctuation_string:
        dt2.loc[k][0] = dt2.loc[k][0].replace(i, '')

In [37]:
# clear out \r \n
for j in range(n1):
    dt1.loc[j][0] = dt1.loc[j][0].replace('\r', ' ')
    dt1.loc[j][0] = dt1.loc[j][0].replace('\n', ' ')

In [38]:
for k in range(n2):
    dt2.loc[k][0] = dt2.loc[k][0].replace('\r', ' ')
    dt2.loc[k][0] = dt2.loc[k][0].replace('\n', ' ')

In [39]:
# split data into train&test parts, test_size = 1000
from sklearn.model_selection import train_test_split
train_CD,test_CD= train_test_split(dt1, test_size=1000)
train_JA,test_JA= train_test_split(dt2, test_size=1000)

Conan_Doyle_class_num = train_CD.shape[0]
Jane_Austin_class_num = train_JA.shape[0]
print("train_data for two books: ",Conan_Doyle_class_num,Jane_Austin_class_num)

train_data for two books:  6023 55188


In [40]:
# calculate p(y)
P_Conan_Doyle = Conan_Doyle_class_num/(Conan_Doyle_class_num+Jane_Austin_class_num)
P_Jane_Austin = Jane_Austin_class_num/(Conan_Doyle_class_num+Jane_Austin_class_num)
print(P_Conan_Doyle,P_Jane_Austin)

0.09839734688209635 0.9016026531179037


In [45]:
# split each paragraph into words

def split(p,dic):
    index = 0   
    start = 0   # the start index of each word
    while index < len(p): 
        start = index   
        while p[index] != " ":
            index += 1   # move the index until as the end of a word
            if index == len(p): 
                break
        if dic.__contains__(p[start:index]):
            dic[p[start:index]] +=1        
        else:
            dic[p[start:index]] = 1
        if index == len(p):
            break
        while p[index] == " ":
            index += 1
            if index ==len(p):
                break
    return dic


In [46]:
# reset_index and split each paragraph into words

train_CD = train_CD.reset_index(drop= True)
test_CD = test_CD.reset_index(drop= True)
train_JA = train_JA.reset_index(drop= True)
test_JA = test_JA.reset_index(drop= True)
dic_A = dict()
dic_B = dict()

for i in range(Conan_Doyle_class_num):
    dic_A = split(str(train_CD.loc[i][0]),dic_A)
for j in range(Jane_Austin_class_num):
    dic_B = split(str(train_JA.loc[j][0]),dic_B)

In [47]:
def words_num_calculate(dic):
    n = 0
    for i in dic:
        n += dic.get(i)
    return n
n_A = words_num_calculate(dic_A)
n_B = words_num_calculate(dic_B)
print(n_A)
print(n_B)
    

91134
766651


In [48]:
print(len(dic_A))
print(len(dic_B))


8728
22328


In [51]:
# This is just an example, i will use the first row data in train_CD
dic_temp = dict()
dic_temp = split(str(train_CD.loc[0][0]),dic_temp)
smooth_fact = 1
sum_p_xc1 = 1
sum_p_xc2 = 1

for i in dic_temp:
    P_xc1 = 0
    P_xc2 = 0
    if(dic_A.get(i)):
        P_xc1 = (dic_A.get(i) + smooth_fact) / n_A
    else:
        P_xc1 = smooth_fact / n_A
    if(dic_B.get(i)):
        P_xc2 = (dic_B.get(i) + smooth_fact) / n_B
    else:
        P_xc2 = smooth_fact / n_B
    sum_p_xc1 *= P_xc1
    sum_p_xc2 *= P_xc2

print(np.log(sum_p_xc1 * P_Conan_Doyle))
print(np.log(sum_p_xc2 * P_Jane_Austin))
print(np.log(sum_p_xc1) > np.log(sum_p_xc2))

-70.37711493175007
-72.11845825125553
True


In [63]:
# my example looks good. So here we go, we calculate accuracy for validation data and test data 
# with hyperparameter smooth_fact

# split to get val_data
train_CD,val_CD= train_test_split(train_CD, test_size=100)
train_JA,val_JA= train_test_split(train_JA, test_size=100)
val_CD = val_CD.reset_index(drop=True)
val_JA = val_JA.reset_index(drop=True)

In [103]:
# pure bayes model p_xc = p_cx * p(x)
def total_accuracy(smooth_fact,x,c,dic_A,dic_B):
    p = x.shape[0]
    j = 0   #j is the right classified number 
    for k in range(p):
        dic_for_para = dict()
        dic_temp = split(str(x.loc[k][0]),dic_for_para)
        sum_p_xc1 = 1
        sum_p_xc2 = 1
        for i in dic_temp:
            P_xc1 = 1
            P_xc2 = 1
            if(dic_A.get(i)):
                P_xc1 = (dic_A.get(i) + smooth_fact) / n_A
            else:
                P_xc1 = smooth_fact / n_A
            if(dic_B.get(i)):
                P_xc2 = (dic_B.get(i) + smooth_fact) / n_B
            else:
                P_xc2 = smooth_fact / n_B

            sum_p_xc1 *= P_xc1
            sum_p_xc2 *= P_xc2
#         P_Conan_Doyle
#         P_Jane_Austin
        P_cx1 = P_Conan_Doyle * sum_p_xc1
        P_cx2 = P_Jane_Austin * sum_p_xc2
#         class-1 means the original target is the belonging to the first book
#         (The Adventures of Sherlock Holmes, by Arthur Conan Doyle)
        if(c == -1):
            if (np.log(P_cx1) > np.log(P_cx2) ):
                j+=1
        else:
            if ( np.log(P_cx1) < np.log(P_cx2) ):
                j+=1
    return j

In [110]:
# on validation dataset
smooth_fact_list = [0.01,0.1,0.5,1,5,10,15,20,50,100]
for s_f in smooth_fact_list:
    c = -1
    p1 = total_accuracy(s_f,val_CD,c,dic_A,dic_B)
    c = 1
    p2 = total_accuracy(s_f,val_JA,c,dic_A,dic_B)
    print("a 'SherlockHolmes snippet' accuracy(smooth_fact {0}) is {1} :".format(s_f,p1/100))
    print("a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact {0}) is {1} :".format(s_f,p2/100))

<ipython-input-103-79056d290e67>:31: RuntimeWarning: divide by zero encountered in log
  if (np.log(P_cx1) > np.log(P_cx2) ):
<ipython-input-103-79056d290e67>:34: RuntimeWarning: divide by zero encountered in log
  if ( np.log(P_cx1) < np.log(P_cx2) ):


a 'SherlockHolmes snippet' accuracy(smooth_fact 0.01) is 0.85 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.01) is 0.98 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 0.1) is 0.85 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.1) is 0.98 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 0.5) is 0.85 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.5) is 0.98 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 1) is 0.85 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 1) is 0.97 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 5) is 0.91 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 5) is 0.83 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 10) is 0.93 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 10) is 0.74 :
a 'SherlockHolmes snippet' accuracy(smooth_fact 15) is 0.92 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 15) is 0.

In [112]:
# on test dataset
dic_test_A = dict()
dic_test_B = dict()
CD_test_distinct_word_num = test_CD.shape[0]
JA_test_distinct_word_num = test_JA.shape[0]

for i in range(CD_test_distinct_word_num):
    dic_test_A = split(str(test_CD.loc[i][0]),dic_test_A)
for j in range(JA_test_distinct_word_num):
    dic_test_B = split(str(test_JA.loc[j][0]),dic_test_B)
    
test_CD_n = words_num_calculate(dic_test_A)
test_JA_n = words_num_calculate(dic_test_B)
print(test_CD_n,test_JA_n)

16432 13723


In [113]:
test_data_num = test_CD.shape[0]
for s_f in smooth_fact_list:
    c = -1
    p1 = total_accuracy(s_f,test_CD,c,dic_A,dic_B)
    c = 1
    p2 = total_accuracy(s_f,test_JA,c,dic_A,dic_B)
    print()
    print("a 'SherlockHolmes snippet' accuracy(smooth_fact {0}) is {1} :".format(s_f,p1/test_data_num))
    print("a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact {0}) is {1} :".format(s_f,p2/test_data_num))

<ipython-input-103-79056d290e67>:31: RuntimeWarning: divide by zero encountered in log
  if (np.log(P_cx1) > np.log(P_cx2) ):
<ipython-input-103-79056d290e67>:34: RuntimeWarning: divide by zero encountered in log
  if ( np.log(P_cx1) < np.log(P_cx2) ):



a 'SherlockHolmes snippet' accuracy(smooth_fact 0.01) is 0.659 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.01) is 0.957 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 0.1) is 0.687 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.1) is 0.946 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 0.5) is 0.725 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 0.5) is 0.927 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 1) is 0.745 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 1) is 0.913 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 5) is 0.809 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 5) is 0.802 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 10) is 0.853 :
a 'The Complete Works of Jane Austen snippet' accuracy(smooth_fact 10) is 0.667 :

a 'SherlockHolmes snippet' accuracy(smooth_fact 15) is 0.872 :
a 'The Complete Works of Jane Austen snippet' accuracy(s

In [ ]:
# from accuracy for val, test dataset of SherlockHolmes snippets and The Complete Works of Jane Austen snippets
# to balance the accuracy on two classes, its better to set hyperparameter smoothing fact between 1 to 5